In [5]:
import numpy as np
from netCDF4 import Dataset

# Define the variables and times
variables = ['lma', 'lai', 'chl']
times = range(13)

def process_file(variable, time_index):
    # Define file paths
    input_file = f"../fitting/transposed_shift_fittings_lwc_time_{time_index:02d}.nc"
    output_file = f"{variable}_aviris_dangermond_clima_fit_time_{time_index:02d}.nc"
    reference_file = f"{variable}_aviris_dangermond_time_{time_index:02d}.nc"

    # Read the input data
    with Dataset(input_file, 'r') as src:
        lat_data = src.variables['lat'][:]
        lon_data = src.variables['lon'][:]
        var_data = src.variables[variable][:]

    # Read the reference file to get the time variable and its attributes
    with Dataset(reference_file, 'r') as ref:
        time_data = ref.variables['time'][:]
        time_units = ref.variables['time'].units
        time_calendar = ref.variables['time'].calendar

    # Reshape var_data to match the dimensions (time, latitude, longitude)
    time_dim = 1
    lat_dim = len(lat_data)
    lon_dim = len(lon_data)
    var_reshaped = var_data.reshape((time_dim, lat_dim, lon_dim))

    # Create the new NetCDF file and define dimensions
    with Dataset(output_file, 'w', format='NETCDF4') as dst:
        dst.createDimension('time', time_dim)
        dst.createDimension('latitude', lat_dim)
        dst.createDimension('longitude', lon_dim)

        # Create time variable
        time = dst.createVariable('time', 'f8', ('time',), fill_value=np.nan)
        time[:] = time_data
        time.units = time_units
        time.calendar = time_calendar

        # Create longitude variable
        longitude = dst.createVariable('longitude', 'f8', ('longitude',), fill_value=np.nan)
        longitude[:] = lon_data
        longitude.standard_name = "longitude"
        longitude.long_name = "longitude"
        longitude.units = "degrees_east"
        longitude.axis = "X"

        # Create latitude variable
        latitude = dst.createVariable('latitude', 'f8', ('latitude',), fill_value=np.nan)
        latitude[:] = lat_data
        latitude.standard_name = "latitude"
        latitude.long_name = "latitude"
        latitude.units = "degrees_north"
        latitude.axis = "Y"

        # Create variable
        var = dst.createVariable(variable, 'f4', ('time', 'latitude', 'longitude',), fill_value=np.nan)
        var[:] = var_reshaped
        var.crs = "EPSG:32610"

    print(f"{variable.upper()} data for time {time_index:02d} successfully extracted and written to {output_file}")

# Loop through each variable and time, and process the corresponding files
for variable in variables:
    for time_index in times:
        process_file(variable, time_index)


LMA data for time 00 successfully extracted and written to lma_aviris_dangermond_clima_fit_time_00.nc
LMA data for time 01 successfully extracted and written to lma_aviris_dangermond_clima_fit_time_01.nc
LMA data for time 02 successfully extracted and written to lma_aviris_dangermond_clima_fit_time_02.nc
LMA data for time 03 successfully extracted and written to lma_aviris_dangermond_clima_fit_time_03.nc
LMA data for time 04 successfully extracted and written to lma_aviris_dangermond_clima_fit_time_04.nc
LMA data for time 05 successfully extracted and written to lma_aviris_dangermond_clima_fit_time_05.nc


KeyboardInterrupt: 

In [1]:
import subprocess
import os

# Define the variables and times
variables = ['lma', 'lai', 'chl']
times = range(13)

# Path to the remap grid file
remap_grid_file = "../California_Vegetation_WHRTYPE_Dangermond/output_latlon.nc"

def regrid_file(variable, time_index):
    input_file = f"{variable}_aviris_dangermond_clima_fit_time_{time_index:02d}.nc"
    output_file = f"{variable}_aviris_dangermond_clima_fit_time_{time_index:02d}_reg.nc"
    
    # Construct the cdo command
    cdo_command = [
        'cdo', f'remapbil,{remap_grid_file}', 
        input_file, 
        output_file
    ]
    
    # Execute the cdo command
    subprocess.run(cdo_command, check=True)
    print(f"Regridded file created: {output_file}")

# Loop through each variable and time, and regrid the corresponding files
for variable in variables:
    for time_index in times:
        regrid_file(variable, time_index)


cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3960431)
Regridded file created: lma_aviris_dangermond_clima_fit_time_00_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3960431)
Regridded file created: lma_aviris_dangermond_clima_fit_time_01_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3960431)
Regridded file created: lma_aviris_dangermond_clima_fit_time_02_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3960431)
Regridded file created: lma_aviris_dangermond_clima_fit_time_03_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with source mask (3960431)
Regridded file created: lma_aviris_dangermond_clima_fit_time_04_reg.nc
cdo    remapbil: Bilinear weights from lonlat (3200x3200) to lonlat (492x458) grid, with s